# 07-02. 심층 신경망

## 은닉층(hidden layer)
- 입력층과 출력층 사이에 존재하는 모든 밀집층
- 비선형적인 데이터에 대한 문제를 해결하기 위해 선형적 계산 결과를 비선형적으로 만들어주는 활성화함수를 각 층마다 가지고 있음
  - 대표적으로 시그모이드 함수와 볼 렐루 함수 등을 사용
- 은닉층의 뉴런 개수에 대한 기준은 없으나 출력층의 뉴런 개수보다 적을 경우 부족한 정보가 전달될 수 있기 때문에 출력층의 뉴런 개수보다는 많은 뉴런을 생성해야함

### 렐루 함수(reLU)
- 입력의 양수일 경우에는 입력값 그대로, 입력이 음수일 경우에는 결과값을 0으로 만드는 함수
- 식: max(0, z)
- 특히 이미지 처리에서 좋은 성능을 보임

## 심층 신경망(deep neural network)
- 2개 이상의 은닉층을 가진 신경망 알고리즘
- 낮은 층위에서는 단순하고 직관적인 특성(ex. 이미지의 수평선, 수직선, 대각선)을 학습하며 높은 층위에서는 복잡하고 추상적인 특성(ex. 특정 사물의 모양)을 학습함

In [1]:
# 케라스 API를 사용하여 패션 MINST 데이터셋 불러오기

from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
# 픽셀값을 전처리 후 세트 분할

from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [3]:
# 시그모이드 함수를 사용하는 은닉층과 소프트맥스 함수를 사용하는 출력층을 생성

dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# 심층 신경망 모델 생성

model = keras.Sequential([dense1, dense2])  # 여러 개의 층을 추가할 때는 출력층을 가장 마지막에 배치해야함
model.summary()  # 층에 대한 유용한 정보 출력

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

- 가장 처음에는 모델의 이름 출력
- 각 층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수 출력
  - 층 이름은 층 생성 시 name 매개변수로 이름을 지정할 수 있으며 지정하지 않을 경우 자동으로 dense로 명명됨
  - 출력 크기: (샘플의 개수, 뉴런의 개수)의 형태로 출력
  - 파리미터 개수: 각 뉴런에 대한 입력값들의 가중치(784 x 100개)과 각 뉴런의 절편(100개)의 합계
- 마지막에는 총 모델 파라미터 개수와 훈련되는 파라미터의 개수, 훈련되지 않는 파라미터의 개수 출력
  - 경사 하강법으로 훈련되지 않는 파라미터가 훈련되지 않는 파라미터 개수에 포함됨

- 샘플의 개수가 None인 이유: 케라스 모델의 fit()은 미니배치 경사 하강법을 사용하기 때문에 샘플의 개수가 배치 크기에 따라 달라질 수 있음   
따라서 샘플의 개수를 None으로 설정해 배치 크기에 관계없이 대응할 수 있도록 함
- 배치 차원: 신경망에 입력되거나 출력되는 배열의 첫 번째 차원, 즉 데이터 샘플의 수

### 층을 추가하는 다른 방법
- Sequentail 클래스 생성자 내부에서 Dense 객체를 직접 생성
- add() 메서드 사용

In [5]:
# Sequential 클래스 생성자 내부에서 Dense 객체를 직접 생성

model = keras.Sequential([keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'), keras.layers.Dense(10, activation='softmax', name='output')], name='패션 MNIST 모델')
model.summary()

Model: "패션 MNIST 모델"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ hidden (Dense)                       │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# add() 메서드 사용

model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# 모델 훈련

model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7544 - loss: 0.7649
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8509 - loss: 0.4213
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8656 - loss: 0.3772
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8710 - loss: 0.3572
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8771 - loss: 0.3411


### Flatten 층
- Flatten 클래스: 배치 차원을 제외한 나머지 입력 차원을 모두 1차원 형태로 펼치는 클래스로 reshape() 없이도 간편하게 1차원 배열을 만들 수 있음
- 입력에 곱해지는 가중치나 절편이 없기 때문에 인공 신경망의 성능에 기여하지는 않지만 입력층과 은닉층 사이에 하나의 층처럼 추가하기 때문에 층으로 취급

In [8]:
# Flatten층을 추가하여 모델 생성

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

- Flatten 층을 통해 입력값의 차원을 짐작할 수 있음
- 입력 테이터에 대한 전처리 과정을 모델에 포함시키고자하는 케라스 API의 철학에 부합

In [9]:
# 훈련 데이터를 다시 준비해 모델 훈련

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7672 - loss: 0.6692
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8536 - loss: 0.4026
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8705 - loss: 0.3518
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8806 - loss: 0.3328
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8851 - loss: 0.3117


- 시그모이드 함수를 사용했을 때에 비해 성능이 약간 향상

In [10]:
# 검증 세트에서의 성능 확인
model.evaluate(val_scaled, val_target)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8606 - loss: 0.4046


[0.403257817029953, 0.8619166612625122]

- ANN 모델(은닉층x)보다 성능 향상

## DNN에서 지정해주어야하는 다양한 하이퍼파라미터
- 은닉층의 개수
- 뉴런의 개수
- 활성화 함수의 종류
- 사용할 층의 종류
- 미니배치의 개수
  - fit() 메서드의 batch_size에서 조절
- 옵티마이저의 종류
  - RMSprop의 학습률


### 옵티마이저(optimizer)
- 케라스에서 제공하는 다양한 종류의 경사 하강법 알고리즘
- compile()의 기본값은 RMSdrop
- compile()의 optimizer 파라미터를 변경해 다른 옵티마이저를 사용할 수 있음

#### SGD 옵티마이저
- 가장 기본적인 옵티마이저
- optimizer='sgd'로 사용 가능
  - sgd = keras.optimizers.SGD()로 선언하고 optimizer=sgd로 사용할 수도 있음
- learning_rate 매개변수로 학습률을 지정
  -
```python
sgd = keras.optimaizers.SGD(learning_rate=0.1)
```
- momentum 매개변수(default:0)를 0 보다 큰 값으로 지정하면 이전 그레이디언트를 가속도처럼 사용하는 모멘텀 최적화를 사용
  - 보통 momentum 매개변수는 0.9 이상을 사용
- nesterov 매개변수를 True로 바꾸면 모멘텀 최적화를 2번 반복하여 구현하는 네스트로프 모멘텀을 사용
  - 대부분의 경우 네스트로프 모멘텀 최적화가 경사 하강법보다 나은 성능 제공
  -
```python
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
```



#### 적응적 학습률(adaptive learning rate)
- 모델이 최적점에 가까이 갈수록 학습률을 낮추어 안정적으로 최적점에 수렴하도록하는 학습률
- Adagard
  -
```python
adagrad = keras.optimizers.Adagard()
model.compile(optimizer=adgrad, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
```
- RMSprop
```python
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
```
- Adam: 모멘텀 최적화와 RMSprop의 장점을 접목한 옵티마이저
- 세 클래스 모두 learning_rate의 기본값이 0.001


In [11]:
# Adam 클래스를 이용해 모델 훈련

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7676 - loss: 0.6819
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8573 - loss: 0.4037
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8738 - loss: 0.3503
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8822 - loss: 0.3225
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8895 - loss: 0.3028


- 기본 RMSprop을 사용했을 때와 거의 같은 성능

In [12]:
# 검증 세트로 검증
model.evaluate(val_scaled, val_target)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8733 - loss: 0.3482


[0.3463119566440582, 0.8736666440963745]

- 기본 RMSprop와 비슷한 성능